### 🚀 Open this Tutorial in Google Colab  
<a target="_blank" href="https://colab.research.google.com/github/amirhszd/MachineLearning4RemoteSensing/blob/main/hw5/hw5_p1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

###Mount Google Drive and Load Data

In this step, you’ll mount your Google Drive so that you can access .npy files directly from it. Then, load your training, validation, and test sets using NumPy. Be sure to update the paths to point to your own data files.

Make sure to set to runtime session to use GPU otherwise your training is going to be slow.

In [1]:
# import files and mount google drive
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# Step 1: Load your .npy files from your Google Drive
# TODO: Replace the paths below with the correct paths to your own files

X_train = np.load('C:/Users/jason/Desktop/git/MachineLearning4RemoteSensing/landis_chlorophyl_regression_train.npy')
y_train = np.load('C:/Users/jason/Desktop/git/MachineLearning4RemoteSensing/landis_chlorophyl_regression_traingt.npy')

X_val = np.load('C:/Users/jason/Desktop/git/MachineLearning4RemoteSensing/landis_chlorophyl_regression_val.npy')
y_val = np.load('C:/Users/jason/Desktop/git/MachineLearning4RemoteSensing/landis_chlorophyl_regression_valgt.npy')

X_test = np.load('C:/Users/jason/Desktop/git/MachineLearning4RemoteSensing/landis_chlorophyl_regression_test.npy')
y_test = np.load('C:/Users/jason/Desktop/git/MachineLearning4RemoteSensing/landis_chlorophyl_regression_testgt.npy')

print("Training set shape:", X_train.shape)

Mounted at /content/drive
Cloning into 'hw5'...
fatal: repository 'https://github.com/amirhszd/MachineLearning4RemoteSensing/tree/main/hw5/' not found


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/landis_chlorophyl_regression_train.npy'

### Standardize the Data & Create Dataset

Here, you will standardize your dataset using StandardScaler. This ensures that each feature has zero mean and unit variance. Remember to fit the scaler only on the training data, and then transform all splits using that same scaler.

You’ll define a custom PyTorch Dataset to wrap your data and make it compatible with DataLoader. After that, instantiate the datasets and set up data loaders with appropriate batch size and parallel loading settings (num_workers, prefetch_factor).

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

# Step 1: Standard scaling based on training data only
# TODO: Scale your validation and test sets using statistics from the training set only
X_train_scaled = # COMPLETE ME
X_val_scaled = # COMPLETE ME
X_test_scaled = # COMPLETE ME

# Step 2: Create custom dataset class
class ChlorophyllDataset(Dataset):
    def __init__(self, X, y):
      # TODO: Convert your numpy arrays to torch tensors with the correct dtype
        self.X = # COMPLETE ME
        self.y = # COMPLETE ME

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Step 3: Instantiate your datasets
train_dataset = ChlorophyllDataset(X_train_scaled, y_train)
val_dataset = ChlorophyllDataset(X_val_scaled, y_val)
test_dataset = ChlorophyllDataset(X_test_scaled, y_test)

# Step 4: Create DataLoaders
# TODO: Set batch_size, num_workers, and prefetch_factor based on your system
from torch.utils.data import DataLoader
batch_size = # COMPLETE ME
num_workers = # COMPLETE ME
prefetch_factor = # COMPLETE ME
train_loader = DataLoader(train_dataset, ...)# COMPLETE ME
val_loader = DataLoader(val_dataset, ...)   # COMPLETE ME
test_loader = DataLoader(test_dataset, ...) # COMPLETE ME

print("train dataset size is:", len(train_dataset))
print("val dataset size is:", len(val_dataset))
print("test dataset size is:", len(test_dataset))

### Define Your 1D Convolutional Model
In this step, you’ll define a custom SpectralCNN model using PyTorch’s nn Module. This model consists of a sequence of 1D convolutional layers with increasing filter sizes and stride to reduce the spectral dimension. The final Linear layer maps the learned features to a single output value for regression.

💡 Note: Since your input has shape [batch_size, 425], you’ll need to change the shape accordingly.

In [ ]:

import torch.nn as nn

class SpectralCNN(nn.Module):
    def __init__(self):
        super(SpectralCNN, self).__init__()
        # 8 layer 1D CNN with kernel size 3 and stride of 2
        # stride 2 lowers the dimensionality spectral dimensionality
        # rely is the activation function introducing nonlinearity
        # the bigger the kernel size the more number of parameters to learn
        self.net = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=3, stride=2), nn.ReLU(),  # [B, 8, 212]
            nn.Conv1d(8, 16, kernel_size=3, stride=2), nn.ReLU(), # [B, 16, 105]
            nn.Conv1d(16, 32, kernel_size=3, stride=2), nn.ReLU(),# [B, 32, 52]
            nn.Conv1d(32, 64, kernel_size=3, stride=2), nn.ReLU(),# [B, 64, 25]
            nn.Conv1d(64, 128, kernel_size=3, stride=2), nn.ReLU(),# [B, 128, 12]
            nn.Conv1d(128, 128, kernel_size=3, stride=2), nn.ReLU(),# [B, 128, 5]
            nn.Conv1d(128, 256, kernel_size=3, stride=2), nn.ReLU(),# [B, 256, 2]
            nn.Conv1d(256, 256, kernel_size=2, stride=1), nn.ReLU(),# [B, 256, 1]
            nn.Flatten(),
            # we are performing regression so output is one value
            nn.Linear(256, 1)
        )

    def forward(self, x):

        # TODO: x is of siz Nx425, however, we need to define the number of channels for our 1D data
        # in this case there is only 1 channel/feature for our 425 input features
        # in 2D space we have N x C_in x H x W.
        # in 1D space we should have N x C_in x Length
        # modify the retun correctly to reflect this
        return # COMPLETE ME

### Define the Model, Optimizer, and Training Configuration

In this section, you will define the loss function for your regression task, instantiate your model, and set up the optimizer of your choice. Make sure to use weight decay to prevent overfitting.

Your training will benefit from learning rate scheduler such as to gradually reduce the learning rate during training. Finally, you’ll move your model to the GPU if available.

💡 Tip: Your learning rate and weight decay are hyperparameters!

In [ ]:
# TODO: Define regression LOSS
criterion = # COMPLETE ME

# instantiating the model
model = SpectralCNN()

# TODO: define optimizer, suggestion is on Adam with weight decay.
# keep in mind weight decay and learning rate are hyperpareters
optimizer = # COMPLETE ME

# TODO: you can define a learning rate scheduler at the time of training
scheduler = # COMPLETE ME

# Optional: use GPU if available
# make sure to change your runtime type to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Using:", device)

### Training Loop and Early Stopping

In this section, you will implement the full training loop for your model. For each epoch, you’ll:
	•	Loop through your training data
	•	Send batches to the correct device
	•	Perform forward and backward passes
	•	Update the model using your optimizer

After training on each epoch, you’ll evaluate your model on the validation set without computing gradients. You’ll track the best-performing model based on validation loss and save it using torch.save(). Finally, you’ll step the learning rate scheduler to gradually reduce the learning rate over time.

Make sure to plot loss for training and validation sets, this will help you see into overfitting and underfitting.

🛑 Don’t forget: only update weights during training, and use torch.no_grad() during validation to save memory and speed up evaluation.

In [ ]:

# Initialize variables for early stopping
best_val_loss = float("inf")
best_epoch = -1

# keepong track of validation loss COMPLETE ME
val_loss_history = []

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    train_losses = []

    # TOOD: keep track of any other regression metric you want per

    for X_batch, y_batch in train_loader:
        # TODO: send input features and reference target to device
        X_batch, y_batch = # COMPLETE ME

        # zeroing out previous step gradients
        # COMPLETE ME

        # send input features and reference target to device
        outputs = model(X_batch)

        # TODO: calculate the loss
        loss = # COMPLETE ME

        # TODO: calculate the gradients by calling backward on the loss
        # COMPLETE ME

        # TODO: take a step by calling step on the optimizer
        # COMPLETE ME

        # TODO: keep score of your training loss and any other metric you see fit for
        # your regression task
        train_losses.append(loss.item())

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            # TODO: Same as training but we are not updating weights
            # COMPLETE ME
            # COMPLETE ME
            # COMPLETE ME
            val_losses.append(val_loss.item())

    # TODO: implementing early stopping here.
    # keep track of the validation loss and save the model parameters
    # when the loss is the lowest for the validation set
    val_loss_mean = # COMPLETE ME
    val_loss_history....            # COMPLETE ME
    if val_loss_mean < best_val_loss:
        # COMPLETE ME
        # COMPLETE ME
        # COMPLETE ME
        torch.save(model.state_dict(), "best_model.pt")

    # TODO: take a step in the scheduler to update the learning rate
    scheduler...# COMPLETE ME

    # Print summary
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {np.mean(train_losses):.4f} | Val Loss: {val_loss_mean:.4f} | LR: {scheduler.get_last_lr()[0]:.6f}")

print(f"\nBest model saved from epoch {best_epoch+1} with Val Loss: {best_val_loss:.4f}")

### Get predictions and plot your results
Get predictions on your test set and plot your regression results for your training and test set.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score

# TODO: we are to check the model's performance on the validation set
# load the "best_model.pt" and pass your test data to it and get predictions
# make sure the model is in eval mode, send the data to device
# note: make sure you handled "shuffle" properly in your validation/testing set
# loading the best model

model.load_state_dict(torch.load("best_model.pt", weights_only=True))
# COMPLETE ME
# COMPLETE ME
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        # COMPLETE ME
        # COMPLETE ME
        # COMPLETE ME

#TODO: plot regression plot and residual plots for your regression task,
# report R2, and MAE along with any other metrics you want
# analyze your results and report on your findings.

I am attaching the test results from Xgboost here for your reference! Your tuned model should outpefrom this performance:

MAE=3.22, R2=0.96

![xgboost](https://github.com/amirhszd/MachineLearning4RemoteSensing/blob/main/hw5/xgboost.png?raw=1)

### Hyperparamter Tuning
This part is all you. You have a base model that is performing somewhat accurately for you. Grab a package and perform hyperparameter tuning on your hyperparameters. Read Below!!!

In [ ]:
# TODO: now that we have a base model that things are working, we perform hyperparameter tuning on validation data

# change the learning rate value
# number of layers
# filter size (the bigger the filter size the more computationally expensive)
# instead of ReLU use leaky Relu or Tanh
# learning weight decay
# using max/average pooling instead of stride 2
# icorporating padding
# you can log the lowest loss (highest accuracy) on your validation set and report that
